# Convolutional Neural Network

### Machine Setup
<p><b>OS</b>: Ubuntu <br></p>

### Import Libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [ ]:
tf.__version__


In [ ]:
datagen = ImageDataGenerator(rescale = 1./255,
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip = True,
                            validation_split = 0.3)


training_set = datagen.flow_from_directory("dataset/archive",
                                        subset='training',
                                        target_size = (64, 64),
                                        batch_size = 32
                                       )
validation_set = datagen.flow_from_directory("dataset/archive",
                                        subset='validation',
                                        target_size = (64, 64),
                                        batch_size = 32
                                       )

    

In [ ]:
class_names = training_set.class_indices
class_names = {class_names[i]:i for i in class_names.keys()}
print(class_names)
train_imgs, labels = next(training_set)

### Model Function

In [ ]:
# a function to automate addition of hidden layers
from keras_visualizer import visualizer
from IPython.display import Image
def cnn_model(hidden_layer=1):
    cnn = tf.keras.models.Sequential()
    
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
    cnn.add(tf.keras.layers.BatchNormalization())
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    cnn.add(tf.keras.layers.Dropout(0.2))
    
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
    cnn.add(tf.keras.layers.BatchNormalization())
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    cnn.add(tf.keras.layers.Dropout(0.2))
    
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
    cnn.add(tf.keras.layers.BatchNormalization())
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    cnn.add(tf.keras.layers.Dropout(0.2))
    
    cnn.add(tf.keras.layers.Flatten())
    for layer in range(hidden_layer):
        cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
    cnn.add(tf.keras.layers.Dense(units=2, activation='softmax'))
    cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return cnn

In [ ]:
# automate the hidden layer count
cnn1 = cnn_model(hidden_layer=1)
cnn2 = cnn_model(hidden_layer=3)
cnn3 = cnn_model(hidden_layer=5)
cnn4 = cnn_model(hidden_layer=7)
cnn5 = cnn_model(hidden_layer=9)


In [ ]:
# Visualize cnn model with 1 hidden layer
history1 = cnn1.fit(x = training_set, validation_data = validation_set, epochs = 50, verbose=0)


In [ ]:
# Visualize cnn model with 3 hidden layer
history2 = cnn2.fit(x = training_set, validation_data = validation_set, epochs = 50, verbose=0)

In [ ]:
# Visualize cnn model with 5 hidden layer
history3 = cnn3.fit(x = training_set, validation_data = validation_set, epochs = 50, verbose=0)


In [ ]:
# Visualize cnn model with 7 hidden layer
history4 = cnn4.fit(x = training_set, validation_data = validation_set, epochs = 50, verbose=0)


In [ ]:
# Visualize cnn model with 9 hidden layer
history5 = cnn5.fit(x = training_set, validation_data = validation_set, epochs = 50, verbose=0)


## Visualize Training History

In [ ]:
historyDf1 = pd.DataFrame.from_dict(history1.history)
historyDf1["epoch"] = np.arange(historyDf1.shape[0])

historyDf2 = pd.DataFrame.from_dict(history2.history)
historyDf2["epoch"] = np.arange(historyDf2.shape[0])

historyDf3 = pd.DataFrame.from_dict(history3.history)
historyDf3["epoch"] = np.arange(historyDf3.shape[0])

historyDf4 = pd.DataFrame.from_dict(history4.history)
historyDf4["epoch"] = np.arange(historyDf4.shape[0])

historyDf5 = pd.DataFrame.from_dict(history5.history)
historyDf5["epoch"] = np.arange(historyDf5.shape[0])


## Model Evaluation

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# visualization function

def viz(title, df, sizex, sizey):
    fig = px.scatter(title=f"Accuracy score of Model with {title} Hidden Layer", 
                 x=[0], y=[0], width=int(sizex), height=int(sizey),
                 labels= {"x":"Epoch","y":"Accuracy"})
    fig.add_trace(go.Scatter(x=df["epoch"], 
                             y=df["accuracy"], 
                             mode="lines",
                             name="Training Accuracy"))
    fig.add_trace(go.Scatter(x=df["epoch"], 
                             y=df["val_accuracy"], 
                             mode="lines", 
                             name="Validation Accuracy"))

    fig.show()

    fig = px.scatter(title=f"Loss score of Model with {title} Hidden Layer", 
                     x=[0], y=[0], width=int(sizex), height=int(sizey),
                     labels= {"x":"Epoch","y":"Loss"})
    fig.add_trace(go.Scatter(x=df["epoch"], 
                             y=df["accuracy"], 
                             mode="lines",
                             name="Training Loss"))
    fig.add_trace(go.Scatter(x=df["epoch"], 
                             y=df["val_loss"], 
                             mode="lines", 
                             name="Validation Loss"))

    fig.show()
    

### 1 Hidden Layer

In [ ]:
historyDf1.tail()

In [ ]:
viz("1", historyDf1, 500, 250)

## 3 Hidden Layer

In [ ]:
historyDf2.tail()

In [ ]:
viz("3", historyDf2, 500, 250)

## 5 Hidden Layer

In [ ]:
historyDf3.tail()

In [ ]:
viz("5", historyDf3, 500, 250)

## 7 Hidden Layer

In [ ]:
historyDf4.tail()

In [ ]:
viz("7", historyDf4, 500, 250)

## 9 Hidden Layer

In [ ]:
historyDf5.tail()

In [ ]:
viz("9", historyDf5, 500, 250)

## Export the best trained model to file for future use

In [ ]:
from keras.models import load_model
cnn2.save("cnnModel_mask-nomask.h5")


# Load Model

A new starting point of testing after training the model.
Model can be used to classify without the need of re-running the training sequence

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import load_model
import numpy as np
import os


loadCNN = load_model("cnnModel_mask-nomask.h5")

2021-09-25 23:39:48.685000: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-25 23:39:48.685116: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-I4FO2JM): /proc/driver/nvidia/version does not exist
2021-09-25 23:39:48.687069: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Pediction Function

In [2]:
def on_button_clicked(_):
    with out:
        out.clear_output()
        try:
            imagepath = f"{os.getcwd()}/img.jpg"
            for key in uploader.value.keys():
                filename = key
            imgByte = uploader.value[filename]['content']
            nparr = np.frombuffer(imgByte, np.uint8)
            img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            RGB_img = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
            imageUploaded = Image.fromarray(RGB_img)
            cv2.imwrite("img.jpg", img_np)
        
            test_image = image.load_img(imagepath, target_size = (64, 64))
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis = 0)
            result = loadCNN.predict(test_image/255.0)

            result = round(result[0][0])
#             print(result)
            if result == 0:
                val = "Without Mask"
                display(HTML(f'<h3>{val}</h1>'))
                imageUploaded.show()
            else:
                val = "With Mask"
                display(HTML(f'<h3>{val}</h1>'))
                imageUploaded.show()
            uploader.value.clear()
            uploader._counter = 0
        except:
            print("Upload image first!!!")

### User Interaction
For model Evaluation

In [3]:
import ipywidgets as widgets
import cv2
from PIL import Image
from IPython.core.display import HTML

uploader = widgets.FileUpload(
    accept="",  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

button = widgets.Button(
    description='Classify',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to let the model classify the uploaded image.',
    icon='robot' # (FontAwesome names without the `fa-` prefix)
)
out = widgets.Output()
button.on_click(on_button_clicked)

In [4]:

display(HTML('<h1>Test the model</h1>'))
display(uploader)
display(widgets.VBox([button,out]))




FileUpload(value={}, description='Upload')